In [1]:
class AlphaVantageCryptoapi:
    '''
    Make a call to Alpha Vantage cryptocurrency API, requesting cryptocurrency info, and allowing me to troubleshoot errors.
        
    Class Members:
    base_url: Required. String. A base_url API call. Defaults to https://www.alphavantage.co/query.

    function_call: String. An API function based on the Alpha Vantage API. Defaults to DIGITAL_CURRENCY_DAILY.

    market_symbol: String. The symbol corresponding to the desired currency market. Defaults to USD.

    crypto_symbol: String. A cryptocurrency symbol.
    
    api_key: String. An api_key as assigned by Alpha Vantage. This member is private and should not be read.
    
    api_url: String. Full url for the api call.

    json: JSON. The data returned by an API call.
    
    df: Pandas.DataFrame. The data returned by an API call converted to a DataFrame.

    error_message: String. Useful for troubleshooting failure.

    '''

    requests = __import__( 'requests' )
    pd = __import__( 'pandas' )
    time = __import__( 'time' )

    # *******************************************************************
    # METHOD __init__()
    # *******************************************************************
    def __init__( 
                self,
                base_url='https://www.alphavantage.co/query',
                function_call='DIGITAL_CURRENCY_DAILY',
                market_symbol='USD',
                crypto_symbol='',
                api_key=''
                ):

        # *******************************************************************
        # INTERNAL METHOD get_api_key()
        # *******************************************************************
        def get_api_key( self, key_to_get, key_path ):
            try:
                # To retrieve the target key, I merely loop through the file, parsing
                # each line on the equal sign. An appropriately formatted line will
                # therefore return two pieces: the first is the key name, and the second
                # is the key value.
                with open( key_path, "r") as log:
                    for sline in log:
                        if sline.split( '=' )[0] == key_to_get:
                            return_val = sline.split( '=' )[1]
                            break

                if return_val == '': raise Exception( f'Failed to get key for {key_to_get}' )
                else: 
                    # We should remove extra spaces and the potential carriage return
                    # at the end.
                    return_val = return_val.strip()

                if return_val[-1] == '\n': return_val = return_val[:-1]

                print( f'Successfully retrieved api key: {key_to_get}' )

                return return_val
            except:
                self.error_message = 'Failure to retrieve api_key'
                return ''
        # *******************************************************************
        # END get_api_key()
        # *******************************************************************

        self.base_url=base_url
        self.function_call=function_call
        self.market_symbol=market_symbol
        self.crypto_symbol=crypto_symbol
        self.__api_key=api_key

        # I'm torn about whether or not to declare the JSON member as an empty dict or a None.
        # json={}
        self.json=None
        self.api_request=None

        # Initializing self.df to a None DataFrame allows a call to df.empty() to test if the
        # DataFrame is empty.
        self.__df = self.pd.DataFrame( None )

        self.error_message=''

        key_path = r'C:\Users\kevin\OneDrive\Documents\Education\Data Analytics\General Assembly\Data Analytics Immersive\_python\python_unit_materials\Classwork'
        key_path = key_path + r'\APIs_in_Python.api_keys.CLASSWORK.txt'

        if self.__api_key == '': self.__api_key = get_api_key( self, 'alpha_vantage_key', key_path )

        # The "{api_key}" will be appended in calls. I don't want to accidentally display
        # the api_key when calling and printing internal members.
        self.api_url = f'{base_url}?function={function_call}&market={market_symbol}&symbol={crypto_symbol}&apikey='

        return None
    # *******************************************************************
    # END _init_()
    # *******************************************************************

    # *******************************************************************
    # METHOD __str__()
    # *******************************************************************
    def __str__( self ):
        return self.crypto_symbol
    # *******************************************************************
    # END __str__()
    # *******************************************************************

    # *******************************************************************
    # PROPERTY api_key
    # *******************************************************************
    def __get_api_key( self ):
        print( 'The api_key should not be read.' )
        return None

    def __set_api_key( self, value ):
        self.__api_key = value
        return None

    def __del_api_key( self ):
        self.__api_key = ''
        return None

    api_key = property( 
                fget = __get_api_key,
                fset = __set_api_key,
                fdel = __del_api_key,
                doc = 'The api_key is the code provided by Alpha Vantage for making calls into its API.'
                )
    # *******************************************************************
    # END PROPERTY api_key
    # *******************************************************************

    # *******************************************************************
    # PROPERTY df
    # *******************************************************************
    def __get_df( self ):
        return self.__df

    def __set_df( self, value ):
        self.__df = value
        return None

    def __del_df( self ):
        self.__df = self.pd.DataFrame( None )
        return None

    df = property( 
                fget = __get_df,
                fset = __set_df,
                fdel = __del_df,
                doc = 'The df property is a wrapper for a Pandas DataFrame.'
                )
    # *******************************************************************
    # END PROPERTY df
    # *******************************************************************

    # *******************************************************************
    # METHOD api_call()
    # *******************************************************************
    def api_call( 
                self,
                crypto_symbol,
                base_url=None,
                function_call=None,
                market_symbol=None,
                api_key=None
                ):
        '''
        Makes an Alpha Vantage API call and returns a status message.
        
        Doc string incomplete.
        '''

        # *******************************************************************
        # INTERNAL METHOD get_json()
        # *******************************************************************
        def get_json(self):
            # if True:
            try:
                self.error_message='Failure creating crypto_call_url'
                # DO NOT APPEND api_key yet.
                self.api_url = f'{self.base_url}?function={self.function_call}&market={self.market_symbol}&symbol={self.crypto_symbol}&apikey='
                # This way we don't expose the api key.
                crypto_call_url = f'{self.api_url}{self.__api_key}'

                self.error_message='Failure calling requests.get()'
                self.api_request = self.requests.get( crypto_call_url )

            # else:
            except:
                # self.api_request = None
                return 'Fail'

            # if True:
            try:
                if self.api_request.status_code != 200:
                    self.error_message = f'Call to requests.get() did not return 200, symbol={self.crypto_symbol},for the following reason: {self.api_request.reason}'
                    # This is a critical error and we should terminate the call.
                    return 'Fail'

                self.error_message = f'Failure on call to <requests.models.Response>.json(), symbol={self.crypto_symbol}'
                self.json = self.api_request.json()

                self.error_message = f'Failed to assign json.keys() to a list variable, symbol={self.crypto_symbol}'
                key_list = list( self.json.keys() )

                # If the crytocurrency call is properly formatted, the first key of the
                # received JSON will be 'Meta Data'.
                self.error_message = f'Failed to retrieve first member of list variable, symbol={self.crypto_symbol}, key_list={key_list}'
                if key_list[0] == 'Meta Data':
                    self.error_message = 'Assign "Time Series (Digital Currency Daily)" data to member json'
                    self.json = self.json['Time Series (Digital Currency Daily)']
                    return 'OK'

                # If the crytocurrency call succeeds, but we have exceeded the threshold of calls per
                # minute or calls per day, the first key of the received JSON will be 'Note'.
                elif key_list[0] == 'Note':
                    self.error_message = f'Note: {self.json}'
                    return f'Note'

                # The crytocurrency call failed for unknown/unexpected reasons. However, this failure
                # is not catastrophic, allowing the calling program to continue
                else:
                    self.error_message = f'Continue: {self.json}'
                    return 'Continue'

            # else:
            except:
                return 'Fail'
        # *******************************************************************
        # END get_json
        # *******************************************************************
        
        # *******************************************************************
        # INTERNAL METHOD convert_json_to_dataframe()
        # *******************************************************************
        def convert_json_to_dataframe(self):

            # if True:
            try:
                # At this point, member json should contain only what we need.
                self.error_message = 'Failure on cast json to pd.DataFrame()'
                self.__df = self.pd.DataFrame( self.json ).T

                # We need to add the symbol to the code.
                self.error_message = f'Failure on call to insert {self.crypto_symbol}'
                self.__df.insert( loc=0, column='Symbol', value=self.crypto_symbol, allow_duplicates=True )

                # The index is a date. We're going to convert that to a column:
                self.error_message = 'Failure on call to DataFrame.reset_index()'
                self.__df.reset_index( inplace=True )

                # I tested the above and found that I want to rename some columns
                # whose names I don't like:
                self.error_message = 'Failure on assignment to dictionary variable'
                dict_renames = { 'index': 'Date Read',
                                '1a. open (USD)': 'Open',
                                '1b. open (USD)': 'OpenB',
                                '2a. high (USD)': 'High',
                                '2b. high (USD)': 'HighB',
                                '3a. low (USD)': 'Low',
                                '3b. low (USD)': 'LowB',
                                '4a. close (USD)': 'Close',
                                '4b. close (USD)': 'CloseB',
                                '5. volume': 'Volume',
                                '6. market cap (USD)': 'Market Cap'
                               }
                self.error_message = 'Failure on call to DataFrame.rename()'
                self.__df.rename( columns=dict_renames, inplace=True )

                # The output was intended, I believe, for two currency types. I'm only using
                # dollars, so I can drop the "B"s.
                self.error_message = 'Failure on creating list variable of columns to drop'
                list_dropcols = [ 'OpenB', 'HighB', 'LowB', 'CloseB' ]
                self.error_message = 'Failure on call to DataFrame.drop()'
                self.__df.drop( labels=list_dropcols, axis=1, inplace=True )

                self.error_message = ''
                return 'OK'

            # else:
            except:
                return 'Fail'

        # *******************************************************************
        # END convert_json_to_dataframe
        # *******************************************************************

        # if True:
        try:
            self.error_message = 'Assigning parameters to member variables.'
            # If the caller passed in values, we'll assign them to our member variables.
            # We are not going to use these parameters; instead, we'll use our member variables.
            if crypto_symbol != None : self.crypto_symbol = crypto_symbol
            if base_url != None : self.base_url = base_url
            if function_call != None : self.function_call = function_call
            if market_symbol != None : self.market_symbol = market_symbol
            if api_key != None : self.__api_key = api_key

            self.error_message = 'Calling internal method get_json()'
            stat = get_json(self)

            if stat == 'OK':

                self.error_message = 'Calling internal method convert_json_to_dataframe()'
                stat = convert_json_to_dataframe(self)

                if stat == 'OK':
                    self.error_message = ''
                    return stat
                else: return stat

            else: return stat

        # else:
        except:
            return 'Fail'

    # *******************************************************************
    # END api_call
    # *******************************************************************

    # *******************************************************************
    # METHOD clear_data()
    # *******************************************************************
    def clear_data( self ):
        self.__df = self.pd.DataFrame( None )
        self.json = None
        self.error_message = ''
        self.api_request=None
        return None
    # *******************************************************************
    # END clear_data()
    # *******************************************************************

    # *******************************************************************
    # METHOD reset()
    # *******************************************************************
    def reset(self):
        # self.__init__(self)
        self.__init__()
        return self
    # *******************************************************************
    # END reset
    # *******************************************************************

    # *******************************************************************
    # METHOD sleep()
    # *******************************************************************
    def sleep(self, value):
        self.time.sleep( value )
    # *******************************************************************
    # END sleep()
    # *******************************************************************

    # *******************************************************************
    # METHOD append()
    # *******************************************************************
    def append( self, masterdf ):
        if type( masterdf ) == self.pd.core.frame.DataFrame:
            if not masterdf.empty:
                return self.pd.concat([ masterdf, self.__df ], axis = 0, ignore_index=False )
            else:
                return self.__df
        else:
            return None
    # *******************************************************************
    # END append()
    # *******************************************************************


<span style="color:red"><i><font size=3>Kevin's comments:</font>
The following cells contain various tests of the `AlphaVantageCryptoapi` Class.
</i></span>

In [2]:
# Initializing the class.
my_avapi = AlphaVantageCryptoapi()

Successfully retrieved api key: alpha_vantage_key


In [3]:
# Demonstrate the wrapped sleep() function.
my_avapi.sleep( 5 )

# Demonstrate the "hidden" api_key attribute.
skey = my_avapi.api_key
print( skey )
# print( my_avapi._AlphaVantageCryptoapi__api_key )

The api_key should not be read.
None


In [4]:
# I wrapped the api_key in a property(), allowing me to prevent the complete deletion
# of the attribute. I should have probably done that for all of the members.
del my_avapi.api_key
stat = my_avapi.api_call( crypto_symbol='ETH' )

# Without the api_key, the api_call will trigger an error.
print( stat )
print( my_avapi.error_message )
print( my_avapi.api_request.status_code )

Continue
Continue: {'Error Message': 'the parameter apikey is invalid or missing. Please claim your free API key on (https://www.alphavantage.co/support/#api-key). It should take less than 20 seconds.'}
200


In [5]:
# The df attribute is also "hidden."
df = my_avapi.df.copy( deep=True )
print( df.empty )

True


In [6]:
# After deleting the api_key value, the only "legit" method to restore it is to
# reset the object.
my_avapi.reset()

Successfully retrieved api key: alpha_vantage_key


In [7]:
# We can "initialize" a master data frame using our class.
masterdf = my_avapi.df.copy( deep=True )

stat = my_avapi.api_call( crypto_symbol='ETH' )
masterdf = my_avapi.append( masterdf )
stat = my_avapi.api_call( crypto_symbol='BTC' )
masterdf = my_avapi.append( masterdf )
masterdf.shape

(2000, 8)

In [8]:
masterdf[997:1003].T

,997,998,999,0,1,2
Date Read,2020-03-14,2020-03-13,2020-03-12,2022-12-06,2022-12-05,2022-12-04
Symbol,ETH,ETH,ETH,BTC,BTC,BTC
Open,134.06000000,107.67000000,194.61000000,16966.35000000,17106.65000000,16885.20000000
High,134.68000000,139.68000000,195.55000000,17102.97000000,17424.25000000,17202.84000000
Low,120.00000000,86.00000000,101.20000000,16965.98000000,16867.00000000,16878.25000000
Close,122.54000000,134.06000000,107.82000000,17045.82000000,16966.35000000,17105.70000000
Volume,1237674.88642000,4663239.92861000,3814533.14046000,16505.89240000,233703.29225000,178619.13387000
Market Cap,1237674.88642000,4663239.92861000,3814533.14046000,16505.89240000,233703.29225000,178619.13387000


In [9]:
my_avapi.df[997:1003].T

,997,998,999
Date Read,2020-03-14,2020-03-13,2020-03-12
Symbol,BTC,BTC,BTC
Open,5576.05000000,4800.01000000,7934.58000000
High,5640.52000000,5955.00000000,7966.17000000
Low,5055.13000000,3782.13000000,4410.00000000
Close,5172.06000000,5578.60000000,4800.00000000
Volume,136910.13597400,402201.67376400,261505.60865300
Market Cap,136910.13597400,402201.67376400,261505.60865300


In [10]:
my_avapi.crypto_symbol

'BTC'

In [11]:
my_avapi.df.columns

Index(['Date Read', 'Symbol', 'Open', 'High', 'Low', 'Close', 'Volume',
       'Market Cap'],
      dtype='object')

<span style="color:red"><i><font size=3>Kevin's comments:</font>
The `AlphaVantageCryptoapi` Class now contains all the code that I had orginally written in several functions and also a loop. I also wrapped some functionality from the "time" module and "pandas" library, eliminating the need to import them separately. This enabled me to write a very clean loop that feels more intuitive. The Class also stores some values that I can inspect should there be a call failure.<br><br>There's no inheritence. And there's still a lot I need to learn about classes and object-oriented programming in Python. But this code feels a lot cleaner than what I had originally written.
</i></span>

In [12]:
# Initialize a new instance of our class.
my_avapi = AlphaVantageCryptoapi()

# We will loop through this list of cryptocurrency symbols, calling 
# the Alpha Vantage API within each loop.
symbols = [ 
            'AINT NO SUCH',
            'ADA',
            'BNB',
            'BTC',
            'BUSD',
            'DOGE',
            'DOT',
            'ETH',
            'LTC',
            'MATIC',
            'XLM',
            'XMR',
            'XRP',
          ]

# We need a default value for our "master" or "container" DataFrame. While looping,
# we will take each DataFrame we have created within the loop and concatenate it to
# this "master" DataFrame. With the first loop, we won't have anything to concatenate
# to, so we will test "all_df.empty" to know whether or not to concatenate.
all_df = my_avapi.df.copy( deep=True )

Successfully retrieved api key: alpha_vantage_key


In [14]:
# Sleep() comes from library "time". We're calling this function to
# ensure that we don't make more than five calls per minute. We are
# about to execute a loop that runs twelve times.
my_avapi.sleep( 90 )

In [15]:
for symbol in symbols:
    stat = my_avapi.api_call( crypto_symbol = symbol )

    if stat == 'OK':
        print( f'API success: symbol={symbol}' )

        # If the api_call() succeeded, we can start lumping all
        # the databases together. The append() method checks 
        # whether or not all_df is empty or not before calling 
        # Pandas concat or Pandas copy.
        all_df = my_avapi.append( all_df )

    else:
        print( f'API fail: {symbol}, {my_avapi.error_message}' )

    # Sleep() comes from library "time". We're calling this function to
    # ensure that we don't make more than five calls per minute.
    my_avapi.sleep( 16 )
    print( f' •• Loop complete' )
    my_avapi.clear_data()

#END_FOR

# We need to rebuild the index. I originally had this within the loop
# as part of the pd.concat() call. But I realized that that was likely
# the most inefficient way to do this.
all_df.reset_index( drop=True, inplace=True )

all_df.shape

API fail: AINT NO SUCH, Continue: {'Error Message': 'Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for DIGITAL_CURRENCY_DAILY.'}
 •• Loop complete
API success: symbol=ADA
 •• Loop complete
API success: symbol=BNB
 •• Loop complete
API success: symbol=BTC
 •• Loop complete
API success: symbol=BUSD
 •• Loop complete
API success: symbol=DOGE
 •• Loop complete
API success: symbol=DOT
 •• Loop complete
API success: symbol=ETH
 •• Loop complete
API success: symbol=LTC
 •• Loop complete
API fail: MATIC, Continue: {'Error Message': 'Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for DIGITAL_CURRENCY_DAILY.'}
 •• Loop complete
API fail: XLM, Continue: {'Error Message': 'Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for DIGITAL_CURRENCY_DAILY.'}
 •• Loop complete
API fail: XMR, Continue: {'Error Message': 'Invalid API call. Plea

NameError: name 'add_df' is not defined

In [16]:
all_df.shape

(8841, 8)